![Iscte Logo](https://www.iscte-iul.pt/assets/images/logo_iscte_detailed.svg)

# Exploração do TAD Lista com implementação de listas ligadas
*Análise da complexidade de algoritmos*
---
**Relatório elaborado no âmbito da Unidade Curricular de Estruturas de Dados e Algoritmos do 1º ano da Licenciatura de Ciência de Dados**

Realizado por:
* 105289 - André Plancha - Turma CDA2
* 105289 - André Plancha - Turma CDA2
* 105289 - André Plancha - Turma CDA2
* 105289 - André Plancha - Turma CDA2

View this in [nbViewer](https://nbviewer.org/github/notPlancha/Trabalho-de-Prog-2/blob/main/Fase%202.ipynb)

# Introdução
Uma lista ligada é uma estrutura de dados que armazena elementos em uma ordem específica.